In [11]:
import yfinance as yf
import pandas as pd
import numpy as np

pd.options.mode.chained_assignment = None

companies_table = pd.read_html('https://en.wikipedia.org/wiki/S%26P/ASX_50')[0]
companies_table


tickers = companies_table['Symbol']
tickers.to_list()
tickers = [ticker + '.AX' for ticker in tickers]
tickers

In [12]:
df = yf.download('RMD.AX', start='2015-01-01', interval='1wk').round(decimals=3)
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-12-29,6.94,6.950,6.87,6.93,6.612,884644
2015-01-05,7.05,7.155,6.94,7.15,6.822,17149927
2015-01-12,7.08,7.430,7.05,7.22,6.889,27227961
2015-01-19,7.35,7.910,7.24,7.80,7.442,54887756
2015-01-26,7.80,8.465,7.80,8.08,7.709,54245873
...,...,...,...,...,...,...
2023-05-15,34.91,35.070,33.63,33.69,33.690,3958578
2023-05-22,33.74,34.240,33.24,33.31,33.310,4372838
2023-05-29,33.32,33.460,32.29,33.00,33.000,6027485


In [13]:
df_copy = df.copy()
df_copy

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-12-29,6.94,6.950,6.87,6.93,6.612,884644
2015-01-05,7.05,7.155,6.94,7.15,6.822,17149927
2015-01-12,7.08,7.430,7.05,7.22,6.889,27227961
2015-01-19,7.35,7.910,7.24,7.80,7.442,54887756
2015-01-26,7.80,8.465,7.80,8.08,7.709,54245873
...,...,...,...,...,...,...
2023-05-15,34.91,35.070,33.63,33.69,33.690,3958578
2023-05-22,33.74,34.240,33.24,33.31,33.310,4372838
2023-05-29,33.32,33.460,32.29,33.00,33.000,6027485


In [14]:
mod_df = df_copy.drop(columns=['Adj Close', 'Volume'])

# EMA40 Calculation
mod_df['EMA20'] = mod_df.Close.ewm(span=20, adjust=False).mean()
mod_df['EMA40'] = mod_df.Close.ewm(span=40, adjust=False).mean()

# MACD and Signal Calculations
exp1 = mod_df.Close.ewm(span=3, adjust=False).mean()
exp2 = mod_df.Close.ewm(span=10, adjust=False).mean()
mod_df['MACD'] = (exp1 - exp2)
mod_df['MACD GRAD'] = mod_df.MACD.diff() 
mod_df['Signal'] = mod_df.MACD.ewm(span=16, adjust=False).mean()
mod_df.dropna(inplace=True)

mod_df

,Open,High,Low,Close,EMA20,EMA40,MACD,MACD GRAD,Signal
Date,,,,,,,,,
2015-01-05,7.05,7.155,6.94,7.15,6.950952,6.940732,0.070000,0.070000,0.008235
2015-01-12,7.08,7.430,7.05,7.22,6.976576,6.954355,0.114545,0.044545,0.020742
2015-01-19,7.35,7.910,7.24,7.80,7.054997,6.995606,0.306901,0.192355,0.054408
2015-01-26,7.80,8.465,7.80,8.08,7.152617,7.048503,0.446782,0.139882,0.100570
2015-02-02,8.06,8.200,7.94,8.13,7.245701,7.101259,0.479299,0.032517,0.145126
...,...,...,...,...,...,...,...,...,...
2023-05-15,34.91,35.070,33.63,33.69,33.074908,32.832400,0.621479,-0.504871,0.363206
2023-05-22,33.74,34.240,33.24,33.31,33.097298,32.855698,0.237534,-0.383945,0.348421
2023-05-29,33.32,33.460,32.29,33.00,33.088032,32.862737,-0.039765,-0.277299,0.302752


In [15]:
mod_df['BuySignal'] = np.where((mod_df.Close > mod_df.EMA40) & (mod_df.MACD > 0) & (mod_df.MACD > mod_df.Signal), 1, 0)
mod_df['SellSignal'] = np.where((mod_df.Close < mod_df.EMA20), 1, 0)
# mod_df['SellSignal'] = np.where((mod_df.MACD < mod_df.Signal), 1, 0)
mod_df['BuyPrice'] = None
mod_df['SellPrice'] = None
# mod_df['SellPriceMaxP'] = None
# mod_df['SellPriceMaxL'] = None
mod_df['PurchaseDate'] = None

mod_df

,Open,High,Low,Close,EMA20,EMA40,MACD,MACD GRAD,Signal,BuySignal,SellSignal,BuyPrice,SellPrice,PurchaseDate
Date,,,,,,,,,,,,,,
2015-01-05,7.05,7.155,6.94,7.15,6.950952,6.940732,0.070000,0.070000,0.008235,1,0,None,None,None
2015-01-12,7.08,7.430,7.05,7.22,6.976576,6.954355,0.114545,0.044545,0.020742,1,0,None,None,None
2015-01-19,7.35,7.910,7.24,7.80,7.054997,6.995606,0.306901,0.192355,0.054408,1,0,None,None,None
2015-01-26,7.80,8.465,7.80,8.08,7.152617,7.048503,0.446782,0.139882,0.100570,1,0,None,None,None
2015-02-02,8.06,8.200,7.94,8.13,7.245701,7.101259,0.479299,0.032517,0.145126,1,0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-15,34.91,35.070,33.63,33.69,33.074908,32.832400,0.621479,-0.504871,0.363206,1,0,None,None,None
2023-05-22,33.74,34.240,33.24,33.31,33.097298,32.855698,0.237534,-0.383945,0.348421,0,0,None,None,None
2023-05-29,33.32,33.460,32.29,33.00,33.088032,32.862737,-0.039765,-0.277299,0.302752,0,1,None,None,None


In [16]:
mod_df_with_signals = mod_df.copy().drop(columns=['EMA40', 'MACD', 'MACD GRAD', 'Signal'])

bought = False

for i in range(0, len(mod_df_with_signals)):
    if not bought:
        if mod_df_with_signals.BuySignal.iloc[i]:
            buyPrice = mod_df_with_signals.Close.iloc[i]
            mod_df_with_signals.BuyPrice.iloc[i] = buyPrice

            purchaseDate = pd.Timestamp(mod_df_with_signals.index.values[i]).strftime('%Y-%m-%d')
            mod_df_with_signals.PurchaseDate.iloc[i] = purchaseDate

            bought = True  

            # priceIndex = 0
            TP = buyPrice * 1.1
            SL = buyPrice * 0.95 
    
    else:
        if not mod_df_with_signals.SellSignal.iloc[i]:
            if ((mod_df_with_signals.Open.iloc[i] >= TP) | (mod_df_with_signals.High.iloc[i] >= TP) | (mod_df_with_signals.Low.iloc[i] >= TP) | (mod_df_with_signals.Close.iloc[i] >= TP)):
                mod_df_with_signals.SellPrice.iloc[i] = TP
                mod_df_with_signals.PurchaseDate.iloc[i] = purchaseDate

                bought = False
        
        else:
            if mod_df_with_signals.Low.iloc[i] <= SL:
                mod_df_with_signals.SellPrice.iloc[i] = SL
                mod_df_with_signals.PurchaseDate.iloc[i] = purchaseDate

                bought = False



mod_df_with_signals




,Open,High,Low,Close,EMA20,BuySignal,SellSignal,BuyPrice,SellPrice,PurchaseDate
Date,,,,,,,,,,
2015-01-05,7.05,7.155,6.94,7.15,6.950952,1,0,7.15,None,2015-01-05
2015-01-12,7.08,7.430,7.05,7.22,6.976576,1,0,None,None,None
2015-01-19,7.35,7.910,7.24,7.80,7.054997,1,0,None,7.865,2015-01-05
2015-01-26,7.80,8.465,7.80,8.08,7.152617,1,0,8.08,None,2015-01-26
2015-02-02,8.06,8.200,7.94,8.13,7.245701,1,0,None,None,None
...,...,...,...,...,...,...,...,...,...,...
2023-05-15,34.91,35.070,33.63,33.69,33.074908,1,0,None,None,None
2023-05-22,33.74,34.240,33.24,33.31,33.097298,0,0,None,None,None
2023-05-29,33.32,33.460,32.29,33.00,33.088032,0,1,None,33.421,2023-05-08


In [17]:
df_grouped_transaction = mod_df_with_signals[~mod_df_with_signals['PurchaseDate'].isnull()]
df_grouped_transaction = df_grouped_transaction[['BuyPrice', 'SellPrice', 'PurchaseDate']].copy()
df_grouped_transaction.set_index('PurchaseDate', inplace=True)
df_grouped_transaction = df_grouped_transaction.groupby(level=0).max()

df_grouped_transaction

,BuyPrice,SellPrice
PurchaseDate,,
2015-01-05,7.15,7.865
2015-01-26,8.08,8.888
2015-03-16,9.38,8.911
2015-07-27,8.0,7.6
2015-10-12,7.73,7.3435
2016-01-18,8.21,7.7995
2016-05-09,7.86,8.646
2016-07-25,9.2,8.74
2016-12-12,8.4,9.24


In [18]:
df_profits = pd.DataFrame(columns=['Profit'])
df_profits['Profit'] = (df_grouped_transaction.SellPrice - df_grouped_transaction.BuyPrice) / df_grouped_transaction.BuyPrice
df_profits

,Profit
PurchaseDate,
2015-01-05,0.1
2015-01-26,0.1
2015-03-16,-0.05
2015-07-27,-0.05
2015-10-12,-0.05
2016-01-18,-0.05
2016-05-09,0.1
2016-07-25,-0.05
2016-12-12,0.1


In [19]:
def analyse(array):
    winrate = len(array[array.Profit > 0]) / len(array)
    aveProfit = array.Profit.mean()
    gain = (array.Profit + 1).cumprod()[-1] if (array.Profit + 1).cumprod()[-1] > 1 else -(1 - (array.Profit + 1).cumprod()[-1])

    return winrate, aveProfit, gain

In [20]:
analyse(df_profits)

(0.5135135135135135, 0.027027027027027056, 2.4293266429991545)